In [2]:
import tensorflow as tf
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.test.is_built_with_cuda()

True

# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
import pickle

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model,Sequential,load_model
from keras.layers import Input, Embedding
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import LSTM
import keras.metrics as metrics
import itertools
from tensorflow.python.keras.utils.data_utils import Sequence
from decimal import Decimal
from keras import backend as K
from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense

# Data Fetching

In [3]:
inp=pd.read_csv("PJ sensor.csv",usecols=[6,7,10,11])
out=pd.read_csv("PJ sensor.csv",usecols=[2,3,4,5,8,9])

In [4]:
inp.head(5)

,Temperature,Relative Humidity,date,Time
0,31,65,191213,175844
1,31,65,191213,175904
2,31,65,191213,175924
3,31,65,191213,175944
4,31,66,191213,180004


In [5]:
out.head(5)

,NO2,O3,CO,SO2,PM2.5,PM10
0,-15.075377,35.714286,1766.233766,148.484848,11.229150,17.323938
1,-20.100503,41.208791,1733.766234,142.424242,9.413914,24.967407
2,-15.075377,43.956044,1711.038961,136.363636,10.350110,56.781536
3,0.000000,57.692308,1688.311688,139.393939,10.488765,30.371115
4,-20.100503,54.945055,1681.818182,130.303030,12.353048,188.026886


In [6]:
inp=np.array(inp)
out=np.array(out)

# Min Max Scaler

In [7]:
from sklearn.preprocessing import MinMaxScaler
import warnings
scaler_obj=MinMaxScaler()
X1=scaler_obj.fit_transform(inp)
Y1=scaler_obj.fit_transform(out)

warnings.filterwarnings(action='ignore', category=UserWarning)

X1=X1[:,np.newaxis,:]
Y1=Y1[:,np.newaxis,:]

In [8]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [13]:
Y1.shape

(1627533, 1, 6)

# Model

In [14]:
model1 = Sequential()
model1.add(keras.Input(shape=(1,4)))
model1.add(tf.keras.layers.LSTM(6,activation="tanh",use_bias=True,kernel_initializer="glorot_uniform",bias_initializer="zeros"))
model1.add(Dense(6))
model1.add(keras.layers.BatchNormalization(axis=-1,momentum=0.99,epsilon=0.001,center=True,scale=True,
                                beta_initializer="zeros",gamma_initializer="ones",
                                moving_mean_initializer="zeros",moving_variance_initializer="ones",trainable=True))
model1.add(keras.layers.ReLU())
model1.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy',metrics=['accuracy','mse','mae',rmse])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 6)                 264       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
batch_normalization_1 (Batch (None, 6)                 24        
_________________________________________________________________
re_lu_1 (ReLU)               (None, 6)                 0         
Total params: 330
Trainable params: 318
Non-trainable params: 12
_________________________________________________________________


In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.25, random_state=42)

model_fit8 = model1.fit(x_train,y_train,batch_size=2048,epochs=50, validation_split=0.1)

Epoch 1/50
537/537 [==============================] - 210s 388ms/step - loss: 1.9613 - accuracy: 0.2974 - mse: 0.1363 - mae: 0.2595 - rmse: 0.3432 - val_loss: 1.7148 - val_accuracy: 0.2882 - val_mse: 0.0826 - val_mae: 0.2064 - val_rmse: 0.2824
Epoch 2/50
537/537 [==============================] - 208s 387ms/step - loss: 1.9172 - accuracy: 0.3024 - mse: 0.1303 - mae: 0.2533 - rmse: 0.3370 - val_loss: 1.8967 - val_accuracy: 0.3055 - val_mse: 0.1274 - val_mae: 0.2507 - val_rmse: 0.3340
Epoch 3/50
537/537 [==============================] - 208s 388ms/step - loss: 1.8828 - accuracy: 0.3063 - mse: 0.1252 - mae: 0.2478 - rmse: 0.3315 - val_loss: 1.8690 - val_accuracy: 0.3089 - val_mse: 0.1234 - val_mae: 0.2460 - val_rmse: 0.3295
Epoch 4/50
537/537 [==============================] - 209s 389ms/step - loss: 1.8542 - accuracy: 0.3103 - mse: 0.1206 - mae: 0.2425 - rmse: 0.3263 - val_loss: 1.8501 - val_accuracy: 0.3131 - val_mse: 0.1194 - val_mae: 0.2412 - val_rmse: 0.3250
Epoch 5/50
537/537 [====

537/537 [==============================] - 208s 388ms/step - loss: 1.4013 - accuracy: 0.3864 - mse: 0.0767 - mae: 0.1734 - rmse: 0.2614 - val_loss: 1.3962 - val_accuracy: 0.3870 - val_mse: 0.0767 - val_mae: 0.1731 - val_rmse: 0.2611
Epoch 35/50
537/537 [==============================] - 204s 379ms/step - loss: 1.3846 - accuracy: 0.3925 - mse: 0.0759 - mae: 0.1724 - rmse: 0.2598 - val_loss: 1.3780 - val_accuracy: 0.3962 - val_mse: 0.0760 - val_mae: 0.1725 - val_rmse: 0.2599
Epoch 36/50
537/537 [==============================] - 204s 381ms/step - loss: 1.3694 - accuracy: 0.3975 - mse: 0.0753 - mae: 0.1715 - rmse: 0.2586 - val_loss: 1.3571 - val_accuracy: 0.4000 - val_mse: 0.0750 - val_mae: 0.1708 - val_rmse: 0.2578
Epoch 37/50
537/537 [==============================] - 209s 390ms/step - loss: 1.3531 - accuracy: 0.4053 - mse: 0.0745 - mae: 0.1706 - rmse: 0.2572 - val_loss: 1.3501 - val_accuracy: 0.4087 - val_mse: 0.0747 - val_mae: 0.1709 - val_rmse: 0.2574
Epoch 38/50
537/537 [===========

# Saving Model as File

In [13]:
model_json = model1.to_json()
with open("lstmpj.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model1.save_weights("lstmpj.h5")
print("Saved model to disk")

Saved model to disk


In [14]:
from keras.models import model_from_json
json_file = open('lstm.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("lstm.h5")
print("Loaded model from disk")
loaded_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse',metrics=['accuracy','mse','mae',rmse,coeff_determination])
print(loaded_model.evaluate(x_train, y_train, verbose=0))

Loaded model from disk
[0.05459023267030716, 0.02124212495982647, 0.05459020659327507, 0.05129491165280342, 0.12296319007873535, -2923.8349609375]


# Error Analysis

In [15]:
_, train_acc = model1.evaluate(x_train, y_train, verbose=0)
_, test_acc = model1.evaluate(x_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

# summarize history for loss
plt.plot(model_fit8.history['loss'])
plt.plot(model_fit8.history['val_loss'])
plt.title('Model Loss',fontweight ='bold',fontsize = 15)
plt.ylabel('Loss',fontweight ='bold',fontsize = 15)
plt.xlabel('Epoch',fontweight ='bold',fontsize = 15)
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# summarize history for accuracy
plt.plot(model_fit8.history['accuracy'])
plt.plot(model_fit8.history['val_accuracy'])
plt.title('Model accuracy',fontweight ='bold',fontsize = 15)
plt.ylabel('Accuracy',fontweight ='bold',fontsize = 15)
plt.xlabel('Epoch',fontweight ='bold',fontsize = 15)
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

ValueError: too many values to unpack (expected 2)